# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from scripts.sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    print(filepath)
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('../../data/song_data')

../../data/song_data


In [5]:
filepath = '/home/fulu/code/postgres-data-modeling/data/song_data/A/A/A/TRAAARJ128F9320760.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# Round the duration to the closest second
df['duration'] = round(df['duration'])

song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = tuple(song_data.values[0])
song_data

('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 270.0)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = tuple(artist_data.values[0])
artist_data

('AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('../../data/log_data')

../../data/log_data


In [12]:
filepath = '/home/fulu/code/postgres-data-modeling/data/log_data/2018/11/2018-11-07-events.json'

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Miami Horror,Logged In,Kate,F,88,Harrell,250.82730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Sometimes,200,1541548876796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
1,The White Stripes,Logged In,Kate,F,89,Harrell,241.86730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,My Doorbell (Album Version),200,1541549126796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
2,Juan Carmona,Logged In,Kate,F,90,Harrell,331.44118,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Panales de Algodon,200,1541549367796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
3,Alison Krauss / Union Station,Logged In,Kate,F,91,Harrell,171.04934,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Restless,200,1541549698796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
4,Bullet For My Valentine,Logged In,Kate,F,92,Harrell,235.65016,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Begging For Mercy,200,1541549869796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Miami Horror,Logged In,Kate,F,88,Harrell,250.82730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Sometimes,200,1541548876796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
1,The White Stripes,Logged In,Kate,F,89,Harrell,241.86730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,My Doorbell (Album Version),200,1541549126796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
2,Juan Carmona,Logged In,Kate,F,90,Harrell,331.44118,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Panales de Algodon,200,1541549367796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
3,Alison Krauss / Union Station,Logged In,Kate,F,91,Harrell,171.04934,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Restless,200,1541549698796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
4,Bullet For My Valentine,Logged In,Kate,F,92,Harrell,235.65016,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Begging For Mercy,200,1541549869796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-07 00:01:16.796
1   2018-11-07 00:05:26.796
2   2018-11-07 00:09:27.796
3   2018-11-07 00:14:58.796
4   2018-11-07 00:17:49.796
Name: ts, dtype: datetime64[ns]

In [16]:
time_data = [(round(dt.timestamp()), dt.hour, dt.day, dt.week, dt.month, dt.year, dt.day_name()) for dt in t]
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_data

[(1541548877, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541549127, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541549368, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541549699, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541549870, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541550105, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541550453, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541550481, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541550651, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541550889, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541551084, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541551209, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541551381, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541551562, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541551775, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541551950, 0, 7, 45, 11, 2018, 'Wednesday'),
 (1541554285, 1, 7, 45, 11, 2018, 'Wednesday'),
 (1541554501, 1, 7, 45, 11, 2018, 'Wednesday'),
 (1541554765, 1, 7, 45, 11, 2018, 'Wednesday'),
 (1541554964, 1, 7, 45, 11, 2018, 'Wednesday'),
 (1541555184, 1, 7, 45, 11, 2018, 'Wedne

In [17]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df

,timestamp,hour,day,week,month,year,weekday
0,1541548877,0,7,45,11,2018,Wednesday
1,1541549127,0,7,45,11,2018,Wednesday
2,1541549368,0,7,45,11,2018,Wednesday
3,1541549699,0,7,45,11,2018,Wednesday
4,1541549870,0,7,45,11,2018,Wednesday
...,...,...,...,...,...,...,...
169,1541624755,21,7,45,11,2018,Wednesday
170,1541629615,22,7,45,11,2018,Wednesday
171,1541629965,22,7,45,11,2018,Wednesday
172,1541632357,23,7,45,11,2018,Wednesday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, tuple(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df[user_df['userId'].astype(bool)]
user_df = user_df.drop_duplicates(subset='userId')
user_df

,userId,firstName,lastName,gender,level
0,97,Kate,Harrell,F,paid
7,63,Ayla,Johnson,F,free
20,8,Kaylee,Summers,F,free
25,9,Wyatt,Scott,M,free
28,89,Kynnedi,Sanchez,F,free
33,50,Ava,Robinson,F,free
34,101,Jayden,Fox,M,free
40,14,Theodore,Harris,M,free
44,10,Sylvie,Cruz,F,free
46,100,Adler,Barrera,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
# insert songplay records
for index, row in df.iterrows():
    # get song_id and artist_id from song and artist tables
    cur.execute(song_select, (row.song, row.artist, round(row.length)))
    result = cur.fetchone()
    (song_id, artist_id) = (result if result else (None, None))

    # uncomment if we only want records with valid song_id and artist_id
    # if song_id is None or artist_id is None:
    #     continue

    # insert songplay record
    songplay_data = (round(row.ts / 1000.0), row.userId, row.level, song_id, \
                     artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.